# Disaster Tweet Classification - High Precision NLP Pipeline

This notebook implements a complete NLP pipeline for classifying disaster-related tweets using:
- **Baseline Model**: TF-IDF + Logistic Regression
- **Advanced Model**: DeBERTa-v3 (Transformer-based)

## Project Structure
1. Data Loading and Exploration
2. Data Preprocessing and Splitting
3. Exploratory Data Analysis (EDA)
4. Text Cleaning and Tokenization
5. Model Training (Baseline + DeBERTa)
6. Model Evaluation and Comparison
7. Final Testing and Deployment

## 1. Installation and Imports

In [ ]:
# Install required packages
!pip install -q transformers datasets scipy scikit-learn torch accelerate


In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, f1_score,
                             accuracy_score, precision_recall_fscore_support,
                             roc_curve, auc)
from sklearn.pipeline import Pipeline
from datasets import Dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                         TrainingArguments, Trainer, DataCollatorWithPadding)

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print("✓ All libraries imported successfully")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")


## 2. Data Loading

Load the disaster tweet dataset. Choose one of the following options:
- **Option 1**: Use Kaggle API (for Google Colab)
- **Option 2**: Load from local file (recommended)

In [ ]:
# OPTION 1: Kaggle API setup (uncomment if using Google Colab)
# !pip install -q kaggle
# from google.colab import files
# files.upload()  # Upload your kaggle.json
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c nlp-getting-started
# !unzip -q nlp-getting-started.zip

# OPTION 2: Load from local file or Google Drive
try:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Specify the path to your file in Google Drive
    file_path = "/content/drive/MyDrive/nlp_train.csv"
    df = pd.read_csv(file_path)
    print(f"✓ Data loaded successfully: {len(df)} rows")
    print(f"✓ Columns: {list(df.columns)}")
except FileNotFoundError:
    print(f"❌ ERROR: The file '{file_path}' was not found.")
    print("Please ensure 'nlp_train.csv' is in the root of your Google Drive.")
    raise
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")
    raise

# Remove rows with missing text or target
df = df.dropna(subset=['text', 'target'])
print(f"✓ After removing NaN: {len(df)} rows")

# Display basic info
print("\nDataset Info:")
print(df.info())
print("\nFirst few rows:")
display(df.head())

## 3. Data Splitting Strategy

**Critical**: Split data BEFORE any preprocessing to prevent data leakage.

Split strategy:
- 10% for final testing (hold-out set)
- From remaining 90%: split into 90% train, 10% validation
- Final split: ~81% train, ~9% validation, ~10% test

In [ ]:
# Split 1: Reserve 10% for final testing (hold-out set)
df_train_full, df_test = train_test_split(
    df,
    test_size=0.10,
    random_state=42,
    stratify=df['target']
)

# Split 2: From remaining 90%, create train (90%) and validation (10%) sets
df_train, df_val = train_test_split(
    df_train_full,
    test_size=0.10,  # 10% of 90% = 9% of total
    random_state=42,
    stratify=df_train_full['target']
)

print("Data Split Summary:")
print(f"Total Dataset:      {len(df):,} rows")
print(f"Training Set:       {len(df_train):,} rows ({len(df_train)/len(df)*100:.1f}%)")
print(f"Validation Set:     {len(df_val):,} rows ({len(df_val)/len(df)*100:.1f}%)")
print(f"Test Set (Hold-out): {len(df_test):,} rows ({len(df_test)/len(df)*100:.1f}%)")

# Verify class distribution
print("\nClass Distribution:")
print(f"Train:      {df_train['target'].value_counts().to_dict()}")
print(f"Validation: {df_val['target'].value_counts().to_dict()}")
print(f"Test:       {df_test['target'].value_counts().to_dict()}")


## 4. Exploratory Data Analysis (EDA)

In [ ]:
# EDA on training data only (to avoid data leakage)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 1. Class Distribution
sns.countplot(x='target', data=df_train, ax=axes[0], palette='viridis')
axes[0].set_title('Class Distribution in Training Set', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Target Class')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Normal (0)', 'Disaster (1)'])

# Add count labels on bars
for p in axes[0].patches:
    axes[0].annotate(f'{int(p.get_height())}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom')

# 2. Tweet Length Distribution
df_train_eda = df_train.copy()
df_train_eda['word_count'] = df_train_eda['text'].apply(lambda x: len(str(x).split()))

sns.histplot(data=df_train_eda, x='word_count', hue='target',
             kde=True, ax=axes[1], palette='viridis', alpha=0.6)
axes[1].set_title('Tweet Length Distribution by Class', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Word Count')
axes[1].set_ylabel('Frequency')
axes[1].legend(['Normal', 'Disaster'])

plt.tight_layout()
plt.show()

# Summary statistics
print("\nWord Count Statistics:")
print(df_train_eda.groupby('target')['word_count'].describe())


## 5. Text Preprocessing

Clean the text data for better model performance:
- Convert to lowercase
- Remove URLs
- Remove HTML tags
- Remove extra whitespace

In [ ]:
def clean_for_bert(text):
    """Clean text for BERT-based models."""
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)                   # Remove HTML tags
    text = re.sub(r'\n', ' ', text)                     # Remove newlines
    text = re.sub(r'\s+', ' ', text)                    # Remove extra spaces
    return text.strip()

# Apply cleaning to all splits (using .copy() to avoid SettingWithCopyWarning)
df_train = df_train.copy()
df_val = df_val.copy()
df_test = df_test.copy()

df_train['clean_text'] = df_train['text'].apply(clean_for_bert)
df_val['clean_text'] = df_val['text'].apply(clean_for_bert)
df_test['clean_text'] = df_test['text'].apply(clean_for_bert)

print("✓ Text cleaning completed")
print("\nExample cleaned texts:")
for i in range(3):
    print(f"\nOriginal: {df_train.iloc[i]['text'][:100]}...")
    print(f"Cleaned:  {df_train.iloc[i]['clean_text'][:100]}...")

# Convert to HuggingFace Dataset format
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "validation": Dataset.from_pandas(df_val),
    "test": Dataset.from_pandas(df_test)
})

print("\n✓ HuggingFace datasets created")
print(dataset)


## 6. Tokenization for DeBERTa

Prepare the data for the DeBERTa transformer model.

In [ ]:
# Initialize tokenizer
model_checkpoint = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

print(f"✓ Tokenizer loaded: {model_checkpoint}")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["clean_text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
columns_to_remove = ["text", "clean_text", "id", "keyword", "location"]
# Only remove columns that actually exist
existing_columns = tokenized_datasets["train"].column_names
columns_to_remove = [col for col in columns_to_remove if col in existing_columns]

# Also remove pandas index column if it exists
if "__index_level_0__" in existing_columns:
    columns_to_remove.append("__index_level_0__")

tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)

# Rename 'target' to 'labels' (PyTorch requirement)
if "target" in tokenized_datasets["train"].column_names:
    tokenized_datasets = tokenized_datasets.rename_column("target", "labels")

# Set format for PyTorch
tokenized_datasets.set_format("torch")

print("\n✓ Tokenization completed")
print("\nFinal dataset structure:")
print(tokenized_datasets)
print(f"\nColumns: {tokenized_datasets['train'].column_names}")


## 7. Model Training: DeBERTa-v3

Train the advanced transformer model with optimized hyperparameters.

In [ ]:
# Initialize model
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)

print(f"✓ Model loaded: {model_checkpoint}")
print(f"✓ Number of parameters: {model.num_parameters():,}")

# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary', zero_division=0
    )
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments (optimized)
training_args = TrainingArguments(
    output_dir="./model_output",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    max_grad_norm=1.0,  # Gradient clipping
    fp16=False,  # Set to True if you have a compatible GPU
    logging_steps=50,
    save_total_limit=2,
)

# Initialize trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\n✓ Trainer initialized")
print("\n" + "="*60)
print("Starting training...")
print("="*60)

# Train the model
trainer.train()

print("\n" + "="*60)
print("✓ Training completed!")
print("="*60)


## 8. Baseline Model: TF-IDF + Logistic Regression

Train a simple baseline model for comparison.

In [ ]:
# Create baseline pipeline
baseline_model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression(solver='liblinear', random_state=42, max_iter=1000))
])

# Train on original text
print("Training baseline model...")
baseline_model.fit(df_train['text'], df_train['target'])
print("✓ Baseline model trained")

# Predict on validation set
y_val_true = df_val['target']
y_pred_baseline = baseline_model.predict(df_val['text'])
y_prob_baseline = baseline_model.predict_proba(df_val['text'])[:, 1]

# Calculate metrics
acc_baseline = accuracy_score(y_val_true, y_pred_baseline)
f1_baseline = f1_score(y_val_true, y_pred_baseline)

print(f"\nBaseline Model Performance (Validation Set):")
print(f"Accuracy: {acc_baseline:.4f}")
print(f"F1-Score: {f1_baseline:.4f}")
print("\nClassification Report:")
print(classification_report(y_val_true, y_pred_baseline,
                           target_names=['Normal', 'Disaster']))


## 9. Model Comparison: DeBERTa vs Baseline

In [ ]:
# Get DeBERTa predictions on validation set
deberta_output = trainer.predict(tokenized_datasets["validation"])
deberta_logits = torch.tensor(deberta_output.predictions)

# Handle NaN/Inf values
if torch.isnan(deberta_logits).any() or torch.isinf(deberta_logits).any():
    print("WARNING: NaN/Inf detected in logits. Replacing with safe values.")
    deberta_logits = torch.nan_to_num(deberta_logits, nan=0.0, posinf=1e10, neginf=-1e10)

# Convert to probabilities
deberta_probs = torch.nn.functional.softmax(deberta_logits, dim=-1).numpy()
y_val_deberta_true = deberta_output.label_ids
y_pred_deberta = np.argmax(deberta_probs, axis=1)
y_prob_deberta = deberta_probs[:, 1]

# Calculate DeBERTa metrics
acc_deberta = accuracy_score(y_val_deberta_true, y_pred_deberta)
f1_deberta = f1_score(y_val_deberta_true, y_pred_deberta)

print("DeBERTa Model Performance (Validation Set):")
print(f"Accuracy: {acc_deberta:.4f}")
print(f"F1-Score: {f1_deberta:.4f}")
print("\nClassification Report:")
print(classification_report(y_val_deberta_true, y_pred_deberta,
                           target_names=['Normal', 'Disaster']))

# Create comparison visualizations
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# 1. Confusion Matrices
cm_baseline = confusion_matrix(y_val_true, y_pred_baseline)
cm_deberta = confusion_matrix(y_val_deberta_true, y_pred_deberta)

sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar=False)
axes[0].set_title(f'Baseline Model\nAccuracy: {acc_baseline:.3f} | F1: {f1_baseline:.3f}',
                 fontsize=12, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_xticklabels(['Normal', 'Disaster'])
axes[0].set_yticklabels(['Normal', 'Disaster'])

sns.heatmap(cm_deberta, annot=True, fmt='d', cmap='Greens', ax=axes[1], cbar=False)
axes[1].set_title(f'DeBERTa Model\nAccuracy: {acc_deberta:.3f} | F1: {f1_deberta:.3f}',
                 fontsize=12, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_xticklabels(['Normal', 'Disaster'])
axes[1].set_yticklabels(['Normal', 'Disaster'])

# 2. ROC Curves
fpr_baseline, tpr_baseline, _ = roc_curve(y_val_true, y_prob_baseline)
auc_baseline = auc(fpr_baseline, tpr_baseline)

fpr_deberta, tpr_deberta, _ = roc_curve(y_val_deberta_true, y_prob_deberta)
auc_deberta = auc(fpr_deberta, tpr_deberta)

axes[2].plot(fpr_baseline, tpr_baseline, linestyle='--',
            label=f'Baseline (AUC = {auc_baseline:.3f})', linewidth=2)
axes[2].plot(fpr_deberta, tpr_deberta,
            label=f'DeBERTa (AUC = {auc_deberta:.3f})', linewidth=2, color='green')
axes[2].plot([0, 1], [0, 1], 'k--', alpha=0.3)
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].set_title('ROC Curve Comparison', fontsize=12, fontweight='bold')
axes[2].legend(loc='lower right')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Performance summary
print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)
print(f"{'Model':<15} {'Accuracy':<12} {'F1-Score':<12} {'AUC':<12}")
print("-"*60)
print(f"{'Baseline':<15} {acc_baseline:<12.4f} {f1_baseline:<12.4f} {auc_baseline:<12.4f}")
print(f"{'DeBERTa':<15} {acc_deberta:<12.4f} {f1_deberta:<12.4f} {auc_deberta:<12.4f}")
print("-"*60)
print(f"{'Improvement':<15} {acc_deberta-acc_baseline:<12.4f} {f1_deberta-f1_baseline:<12.4f} {auc_deberta-auc_baseline:<12.4f}")
print("="*60)


## 10. Final Testing on Hold-out Set

Evaluate the best model on the test set that was never used during training.

In [ ]:
print("Evaluating on hold-out test set...")

# Test DeBERTa model
test_output = trainer.predict(tokenized_datasets["test"])
test_preds = np.argmax(test_output.predictions, axis=-1)
test_labels = test_output.label_ids

# Calculate metrics
test_acc = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds)

print("\n" + "="*60)
print("FINAL TEST SET RESULTS (DeBERTa)")
print("="*60)
print(f"Accuracy: {test_acc:.4f}")
print(f"F1-Score: {test_f1:.4f}")
print("\nDetailed Classification Report:")
print(classification_report(test_labels, test_preds,
                           target_names=['Normal (0)', 'Disaster (1)']))

# Confusion matrix
cm_test = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Greens', cbar=False)
plt.title(f'Test Set Confusion Matrix\nAccuracy: {test_acc:.3f} | F1: {test_f1:.3f}',
         fontsize=14, fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks([0.5, 1.5], ['Normal', 'Disaster'])
plt.yticks([0.5, 1.5], ['Normal', 'Disaster'])
plt.tight_layout()
plt.show()

# Show some misclassified examples
print("\nSample Misclassified Examples:")
print("="*60)
misclassified = np.where(test_preds != test_labels)[0]
for i, idx in enumerate(misclassified[:5]):
    text = tokenizer.decode(tokenized_datasets["test"][int(idx)]["input_ids"],
                           skip_special_tokens=True)
    print(f"\nExample {i+1}:")
    print(f"Text: {text}")
    print(f"True Label: {test_labels[idx]} ({'Disaster' if test_labels[idx]==1 else 'Normal'})")
    print(f"Predicted: {test_preds[idx]} ({'Disaster' if test_preds[idx]==1 else 'Normal'})")
    print("-"*60)


## 11. Prediction Function

Create a function to classify new tweets.

In [ ]:
def predict_tweet(text):
    """
    Predict whether a tweet is about a disaster or not.

    Args:
        text (str): Tweet text to classify

    Returns:
        str: Classification result with confidence
    """
    if not text or len(text.strip()) == 0:
        return "⚠️ Invalid input: empty text"

    # Set model to evaluation mode
    model.eval()

    # Clean the text
    cleaned_text = clean_for_bert(text)
    if len(cleaned_text.strip()) == 0:
        cleaned_text = text  # Fall back to original if cleaning removes everything

    # Tokenize
    inputs = tokenizer(
        cleaned_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        # Check for NaN
        if torch.isnan(logits).any():
            return "⚠️ Model error: NaN values detected. Please retrain the model."

        # Get probabilities
        probs = torch.nn.functional.softmax(logits, dim=1)

    # Get prediction
    confidence, label_idx = torch.max(probs, dim=1)
    label = "🚨 DISASTER" if label_idx.item() == 1 else "✅ NORMAL"

    return f"{label} (Confidence: {confidence.item()*100:.1f}%)"

# Test the function
print("Testing prediction function:")
print("="*60)

test_tweets = [
    "Forest fire near my house!",
    "I am eating a delicious pizza.",
    "Earthquake strikes the city, buildings collapsed!",
    "Beautiful sunset at the beach today",
    "Flood warning issued for coastal areas"
]

for tweet in test_tweets:
    result = predict_tweet(tweet)
    print(f"Tweet: '{tweet}'")
    print(f"Prediction: {result}")
    print("-"*60)


## 12. Save Model for Deployment

Save the trained model and tokenizer for future use.

In [ ]:
# Save model and tokenizer
model_save_path = "./saved_disaster_model"

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"✓ Model saved to: {model_save_path}")

# Create a zip file for easy download
!zip -r saved_disaster_model.zip ./saved_disaster_model

print("\n✓ Model archived: saved_disaster_model.zip")
print("\nTo load the model later:")
print("""from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./saved_disaster_model')
tokenizer = AutoTokenizer.from_pretrained('./saved_disaster_model')""")


## Summary

This notebook demonstrated a complete NLP pipeline for disaster tweet classification:

### Key Achievements:
1. ✅ Proper data splitting to prevent leakage
2. ✅ Comprehensive EDA and text preprocessing
3. ✅ Baseline model (TF-IDF + LogReg) for comparison
4. ✅ Advanced model (DeBERTa-v3) with optimized hyperparameters
5. ✅ Robust evaluation on hold-out test set
6. ✅ Deployment-ready prediction function

### Next Steps:
- Fine-tune hyperparameters further
- Try other transformer models (RoBERTa, BERT, etc.)
- Implement ensemble methods
- Deploy as a web service or API

### Model Files:
- Model and tokenizer saved in `./saved_disaster_model`
- Download `saved_disaster_model.zip` for deployment